In [38]:
#Метод K-ближайших соседей
import math
import random
import numpy as np
import time
from sklearn.datasets import load_iris
from sklearn.datasets import load_wine
from sklearn.datasets import load_breast_cancer
from sklearn.datasets import load_digits
import numba


#Главная функция (с возможностями sklearn)
def mainSk():
    pass


#Главная функция (самописная)
def main():
    start_time = time.time()
    print('Start load datasets...')

    #Обрабатываем первый датасет
    analyseDataset(load_iris())
    print('End analyse iris-dataset')
    analyseDataset(load_wine())
    print('End analyse wine-dataset')
    analyseDataset(load_breast_cancer())
    print('End analyse breast_cancer-dataset')
    analyseDataset(load_digits())
    print('End analyse digits-dataset')

    print('End load datasets')
    print("--- %s seconds ---" % (time.time() - start_time))



#Метод анализирования КПД путем изменения метрик и числа ближайших соседей
def analyseDataset(dataset):

    #Выделяем классы в отдельные списки (НО ЭТО НЕ ТОЧНО)
    dataList = np.array(dataset.data)
    targetList = np.array(dataset.target)

    #Берем <=30% данных
    sizeDataset = len(dataset.data)
    sizeTestDataList = math.floor(sizeDataset * 0.3)
    arrayIndex = np.arange(sizeDataset)
    np.random.shuffle(arrayIndex)
    testDataList = dataList[arrayIndex[:sizeTestDataList]]       #30% данных
    sourceDataList = dataList[arrayIndex[sizeTestDataList:]]     #70% данных
    testTargetList = targetList[arrayIndex[:sizeTestDataList]]   #30% классов данных
    sourceTargetList = targetList[arrayIndex[sizeTestDataList:]] #70% классов данных
    
    #Нормализуем данные
    dataList = normalize(dataList)
    
    #Меняем число ближайших сосдей
    arrayOfPercent = [] #Массив с окончательными ответами (массив с процентами правильности, названием метрики и числом соседей)
    countMaxNeighbour = 10
    countMinNeighbour = 3
    arrayOfThreads = []
    for n in range(countMinNeighbour, countMaxNeighbour + 1):
        answerArray = countNearestNeighbors(sourceDataList, sourceTargetList, testDataList, metricL1, n)
        arrayOfPercent.append([getPercentOfTrue(answerArray, testTargetList), 'metric L1', n])
        answerArray = countNearestNeighbors(sourceDataList, sourceTargetList, testDataList, metricL2, n)
        arrayOfPercent.append([getPercentOfTrue(answerArray, testTargetList), 'metric L2', n])
        answerArray = countNearestNeighbors(sourceDataList, sourceTargetList, testDataList, metricLInfinity, n)
        arrayOfPercent.append([getPercentOfTrue(answerArray, testTargetList), 'metric Linfinity', n])
        #thread1 = Thread(target=functionForThread, args=(sourceDataList, sourceTargetList, testDataList, metricL1, n, testTargetList, "metric L1",))
        #arrayOfThreads.append(thread1)
        #thread1.start()
        #thread2 = Thread(target=functionForThread, args=(sourceDataList, sourceTargetList, testDataList, metricL2, n, testTargetList, "metric L2",))
        #arrayOfThreads.append(thread2)
        #thread2.start()
        #thread3 = Thread(target=functionForThread, args=(sourceDataList, sourceTargetList, testDataList, metricLInfinity, n, testTargetList, "metric LInfinity",))
        #arrayOfThreads.append(thread3)
        #thread3.start()
       
    
    #Ждем выполнение потоков
    #for t in arrayOfThreads:
    #    t.join()

    #Анализируем данные и выводим их на экран
    maxPercent = 0
    for i in arrayOfPercent:
        if (i[0] >= maxPercent):
            maxPercent = i[0]
    print("### Статистика по датасету ###")
    print("Максимальный процент правильности = ", round(maxPercent*100, 2), "% имеют способы: ")
    for i in arrayOfPercent:
        if (i[0] == maxPercent):
            print(i[1], ", число соседей = ", i[2])
            
            

In [39]:
#Метод нормализации тестовых данных
#На вход принимается 1 спискок: датасет, который необходимо нормализовать
@numba.njit
def normalize(dataset):
    
    #В поисках минимума и максимума
    valueMax = dataset.max()
    valueMin = dataset.min()

    return [[(elem - valueMin) / (valueMax - valueMin) for elem in row] for row in dataset]

In [40]:
#Метрика l1
@numba.njit
def metricL1(dataFromSource, dataTest):
    return np.abs(dataFromSource - dataTest).sum()

@numba.njit
#Метрика l2
def metricL2(dataFromSource, dataTest):
    return np.sqrt(np.square(dataFromSource - dataTest).sum())


#Метрика l бесконечность
@numba.njit
def metricLInfinity(dataFromSource, dataTest):
    return np.abs(dataFromSource - dataTest).max()

In [41]:
#Метод близжайших соседей
#На вход принимает 5 параметров: 
#sourceData - нормализированный список МАРКИРОВАННЫХ данных
#sourceTarget - список классов МАРКИРОВАННЫХ данных
#testData - нормализированный список ТЕСТИРУЕМЫХ (ПРОГНОЗИРУЕМЫХ) данных, 
#functionMetric - функцию используемой метрики
#N - число ближайших соседей
#Функция возвращает список СПРОГНОЗИРОВАННЫХ меток для каждого элемента testData
def countNearestNeighbors(sourceData, sourceTarget, testData, functionMetric, N):
    answerArray = np.zeros(len(testData)) #Массив с СПРОГНОЗИРОВАННЫМИ метками
    
    #Считаем ближайшие N классов для каждого тестового элемента
    dtype = [('data', float), ('target', float)]
    arrayWithDiff = np.sort(np.array([[(functionMetric(test, source), sourceTarget[index]) for index, source in enumerate(sourceData)] for test in testData], dtype=dtype), order='data')['target'][:,:N,] #Рассчет метрик
    
    #Считаем какой самый часто встречающийся сосед
    targetUnicList = set(sourceTarget)
    for index, testElem in enumerate(arrayWithDiff):
        countOfTarget = np.zeros(len(targetUnicList))
        for sourceT in testElem:
            for targetU in targetUnicList:
                if (sourceT == targetU):
                    countOfTarget[targetU] = countOfTarget[targetU] + 1
                    break
            
        #Проверка на одинаковое число максимумов (какой элемент ближе)
        maxCount = countOfTarget.max()
        for nowElem in testElem:
            if (countOfTarget[int(nowElem)] == maxCount):
                answerArray[index] = nowElem
                break
                
    return answerArray

In [42]:
#Метод подсчета процента правильности алгоритма
#На вход принимает 2 параметра:
#answerArray - рассчитаный список target-ов
#trueTargetList - истинный список таргетов
#Метод возвращает число - процент правильности определения класса
@numba.njit
def getPercentOfTrue(answerArray, trueTargetList):
    countTrue = 0
    countAll = len(answerArray)
    for i in range(countAll):
        if (answerArray[i] == trueTargetList[i]):
            countTrue = countTrue + 1
    return countTrue/countAll

In [43]:
#Метод для передачи в поток
def functionForThread(sourceData, sorceTarget, testData, metric, n, testTrueTargetList, nameMetric):
    answerArray = countNearestNeighbors(sourceData, sourceTarget, testData, metric, n)
    arrayOfPercent.append([getPercentOfTrue(answerArray, testTrueTargetList), nameMetric, n])
    print("поток ", nameMetric, " ", n, " отработал")

In [44]:
main()

Start load datasets...


TypingError: Failed in nopython mode pipeline (step: nopython frontend)
[1mUntyped global name 'countNearestNeighbors':[0m [1m[1mCannot determine Numba type of <class 'function'>[0m
[1m
File "C:\Users\kuzin\AppData\Local\Temp\ipykernel_16916\2911464067.py", line 65:[0m
[1m<source missing, REPL/exec in use?>[0m
[0m 

This error may have been caused by the following argument(s):
- argument 0: [1mCannot determine Numba type of <class 'sklearn.utils.Bunch'>[0m
